# Benchmarcks and plots of SVM PPO step envs

In [ ]:
import numpy as np
import h5py

import matplotlib
import matplotlib.pyplot as plt
import itertools
from matplotlib import cm
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
%load_ext autoreload
%autoreload 2
from IPython import get_ipython

In [ ]:
## All the data stored in the hdf5 file
f = h5py.File('runs_step_envs/run_17.hdf5', 'r')

list(f.keys())

In [ ]:
## Read file and print its info and hyperparams
for k in f['info'].attrs.keys():
    print(k, ' : ', f['info'].attrs[k])

for k in f['hyperparams'].attrs.keys():
    print(k, ' : ', f['hyperparams'].attrs[k])

In [ ]:
## Example of data stored and number of episodes
num_ep = len(f['rewards'].keys())
print('The number of episodes is then:', num_ep)

In [ ]:
## Score data analysis
scores = np.zeros(len(f['rewards'].keys()))
for i in range(len(f['rewards'].keys())):
    scores[i] = np.mean([np.sum(rew_traj) for rew_traj in (f['rewards'][f'rew_ep_{i}'][:])])
    
## Max and min
index_max = np.argmax(scores)
index_min = np.argmin(scores)
print(f'The max score is at episode {index_max} and is equal to {scores[index_max]}')
print(f'The min score is at episode {index_min} and is equal to {scores[index_min]}')
    
## Plot scores
plt.rcParams.update({'font.size': 11})
plt.grid(True)
plt.ylabel('Score')
plt.xlabel('Episode \#')
plt.plot(np.arange(len(scores)), scores)
plt.savefig('./graphs/scores_ppo_step_den.pdf', format='pdf', dpi=1000)

In [ ]:
## Energies data analysis
en_per_ep = np.zeros(num_ep)
for i in range(num_ep):
    en_ep = f['energies'][f'en_ep_{i}'][:]
    en_per_ep[i] = np.mean([min(en_traj) for en_traj in en_ep])
    
## Max and min
index_min = np.argmin(en_per_ep)
index_max = np.argmax(en_per_ep)
print(f'The min en (BEST CASE) is at episode {index_min} and is equal to {en_per_ep[index_min]}')
print(f['energies'][f'en_ep_{index_min}'][:])
print(f'The max en (WORST CASE) is at episode {index_max} and is equal to {en_per_ep[index_max]}')
print(f['energies'][f'en_ep_{index_max}'][:])

## Plot energies over episodes 
plt.rcParams.update({'font.size': 11})
plt.grid(True)
plt.ylabel('Mean of last energies of trajectories for Episode (K)')
plt.xlabel('Episode \#')
plt.plot(np.arange(len(en_per_ep)), en_per_ep)
plt.savefig('./graphs/energies_ppo_step_den.pdf', format='pdf', dpi=1000)

In [ ]:
## Pri dims data analysis
pri_per_ep = np.zeros(num_ep)
for i in range(num_ep):
    pri_ep = f['princip_dims'][f'pri_dim_ep_{i}'][:]
    pri_per_ep[i] = np.mean([max(pri_traj) for pri_traj in pri_ep])
    
## Max and min
index_min = np.argmin(pri_per_ep)
index_max = np.argmax(pri_per_ep)
print(f'The biggest pri dim (BEST CASE) is at episode {index_max} and is equal to {pri_per_ep[index_max]}')
print(f['energies'][f'en_ep_{index_min}'][:])
print(f'The smallest pri dim (WORST CASE) is at episode {index_min} and is equal to {pri_per_ep[index_min]}')
print(f['energies'][f'en_ep_{index_min}'][:])

## Plot pri dim over episode
plt.rcParams.update({'font.size': 11})
plt.grid(True)
plt.ylabel('Mean of the last principal size of trajectories for episode')
plt.xlabel('Episode \#')
plt.plot(np.arange(len(pri_per_ep)), pri_per_ep)
plt.savefig('./graphs/pri_dim_ppo_step_den.pdf', format='pdf', dpi=1000)

In [ ]:
## Rewards of episode with max score
rew_max = f['rewards'][f'rew_ep_{index_max}'][:]
print(rew_max)
rew_of_each_trajs = [np.sum(rew_traj) for rew_traj in(rew_max)]

# Plot rew of episode with max score
plt.rcParams.update({'font.size': 11})
plt.plot(np.arange(len(rew_max)), rew_of_each_trajs)
plt.ylabel('Reward')
plt.xlabel('# Traj')

print('This episode collected a score of = ', np.mean(rew_of_each_trajs))

In [ ]:
## Plot energies of episode with max score
en_max = f['energies'][f'en_ep_{index_max}'][:]
print(en_max)

# Plot energies of episode with max score
plt.rcParams.update({'font.size': 11})
plt.plot(np.arange(len(en_max)), [en_max[i,-1] for i in range(len(en_max))])
plt.ylabel('Last Energy (K)')
plt.xlabel('# Traj')

print('The last energy reached is = ',en_max[-1,-1])

In [ ]:
## Plot princip dim of episode with max score
pri_max = f['princip_dims'][f'pri_dim_ep_{index_max}'][:]
print(pri_max)

# Plot pri dim of episode with max score
plt.rcParams.update({'font.size': 11})
plt.plot(np.arange(len(pri_max)), [pri_max[i,-1] for i in range(len(pri_max))])
plt.ylabel('Last Prinicip dim')
plt.xlabel('# Traj')

print('The last princip dim reached is = ', pri_max[-1,-1])

In [ ]:
## Sigmas analysis (comparing with random and the one collected in the epsidoe with max score)

sigmas_random = np.loadtxt('sigmas_random.out')
sigmas_random_squared = sigmas_random[:,0]**2 + sigmas_random[:,1]**2 + sigmas_random[:,2]**2

sigmas_rl = f['sigmas'][f'sigmas_ep_{index_max}'][:]
sigmas_rl = sigmas_rl*109/2 + 111/2
print(sigmas_rl.shape)
sigmas_rl_last = sigmas_rl[-1]
print(sigmas_rl_last)
sigmas_rl_squared = sigmas_rl_last[:,0]**2 + sigmas_rl_last[:,1]**2 + sigmas_rl_last[:,2]**2

# Plot sigmas dist of episode with max score
plt.rcParams.update({'font.size': 11})
plt.grid(True)
plt.ylabel('Counts')
plt.xlabel('$\\alpha_{12}^2 + \\alpha_{13}^2 + \\alpha_{23}^2$')
plt.hist(sigmas_random_squared, bins=20, alpha=0.5)
plt.hist(sigmas_rl_squared, bins=20, alpha=0.5)
plt.savefig('./graphs/sigmas_max_ppo_step_den.pdf', format='pdf', dpi=1000)

In [ ]:
## Rewards of episode with min score
rew_min = f['rewards'][f'rew_ep_{index_min}'][:]
print(rew_min)
rew_of_each_trajs = [np.sum(rew_traj) for rew_traj in(rew_min)]

# Plot rew of episode with min score
plt.rcParams.update({'font.size': 11})
plt.plot(np.arange(len(rew_min)), rew_of_each_trajs)
plt.ylabel('Reward')
plt.xlabel('# Traj')

print('This episode collected a score of = ', np.mean(rew_of_each_trajs))

In [ ]:
## Plot energies of episode with min score
en_min = f['energies'][f'en_ep_{index_min}'][:]
print(en_min)

# Plot energies of episode with min score
plt.rcParams.update({'font.size': 11})
plt.plot(np.arange(len(en_min)), [en_min[i,-1] for i in range(len(en_min))])
plt.ylabel('Last Energy (K)')
plt.xlabel('# Traj')

print('The last energy reached is = ',en_min[-1,-1])

In [ ]:
## Plot princip dim of episode with max score
pri_min = f['princip_dims'][f'pri_dim_ep_{index_min}'][:]
print(pri_min)

# Plot pri dims of episode with max score
plt.rcParams.update({'font.size': 11})
plt.plot(np.arange(len(pri_min)), [pri_min[i,-1] for i in range(len(pri_min))])
plt.ylabel('Last Prinicip dim')
plt.xlabel('# Traj')

print('The last princip dim reached is = ', pri_min[-1,-1])

In [ ]:
## Sigmas analysis (comparing with random and the one collected in the epsidoe with min score)

sigmas_random = np.loadtxt('sigmas_random.out')
sigmas_random_squared = sigmas_random[:,0]**2 + sigmas_random[:,1]**2 + sigmas_random[:,2]**2

sigmas_rl = f['sigmas'][f'sigmas_ep_{index_min}'][:]
sigmas_rl = sigmas_rl*109/2 + 111/2
sigmas_rl_last = sigmas_rl[-1]
print(sigmas_rl_last)
sigmas_rl_squared = sigmas_rl_last[:,0]**2 + sigmas_rl_last[:,1]**2 + sigmas_rl_last[:,2]**2

# Plot sigmas dist of episode with min score
plt.rcParams.update({'font.size': 11})
plt.grid(True)
plt.ylabel('Counts')
plt.xlabel('$\\alpha_{12}^2 + \\alpha_{13}^2 + \\alpha_{23}^2$')
plt.hist(sigmas_random_squared, bins=20, alpha=0.5)
plt.hist(sigmas_rl_squared, bins=20, alpha=0.5)
plt.savefig('./graphs/sigmas_min_ppo_step_den.pdf', format='pdf', dpi=1000)